# Calibrate 4D STEM rotation and handedness

This notebook uses the defocus method together with a digital twin to adjust the rotation and handedness of a 4D STEM setup. The detector images in a strongly defocused 4D STEM dataset are superimposed. If the parameters of a digital twin match the microscope parameters, the images are correctly superimposed to generate an image of the specimen. If not, the superimposed image is blurred.

This notebook requires `libertem[bqplot]`, `jupyter-ui-poll` and `ipywidgets`.

In [1]:
import numpy as np
import ipywidgets
from jupyter_ui_poll import ui_events
from skimage.measure import blur_effect

from libertem.api import Context
from libertem.udf.sum import SumUDF
from libertem.viz.bqp import BQLive2DPlot

from microscope_calibration.udf.stem_overfocus import OverfocusUDF
from microscope_calibration.common.model import Parameters4DSTEM, PixelYX, Model4DSTEM
from microscope_calibration.util.optimize import make_overfocus_loss_function, optimize

In [2]:
# Default LiberTEM Context with best performance for offline data.
# It may not work within Apptainer images if /dev/shm is not available.
# since this is required for Python multiprocessing.
# ctx = Context()

# Compatible with running inside an Apptainer image
# ctx = Context.make_with('threads')
# Debugging
ctx = Context.make_with('inline')

## Load data

The demo file `fourdstem_overfocused.npy` can be generated with the `generate.ipynb` notebook. You can specify use your own dataset in a format that LiberTEM supports. See https://libertem.github.io/LiberTEM/formats.html#supported-formats for supported file formats.

For live processing see https://libertem.github.io/LiberTEM-live/

In [3]:
# Use simulated test dataset
ds = ctx.load('auto', path='fourdstem_overfocused.npy')

# Use the dataset from https://doi.org/10.5281/zenodo.10418769
# Adjust path as needed
#ds = ctx.load('auto', path=r'C:/Users/weber/Nextcloud3/ER-C-Data/adhoc/livecalibration/20221025_154811/rotation_calibration.hdr')

## Set starting values

Set sensible starting values based on your known microscope parameters. Note that the values here don't match the values for fourdstem_overfocused.npy or the measured data so that the adjustment procedure can be tested.

In [4]:
overfocus_params = Parameters4DSTEM(
    overfocus=0.0015,  # m
    scan_pixel_pitch=0.000001,  # m
    scan_center=PixelYX(
        y=ds.shape.nav[0] / 2,
        x=ds.shape.nav[1] / 2,
    ),
    camera_length=0.15,  # m,
    detector_pixel_pitch=0.000050,  # m
    detector_center=PixelYX(
        y=ds.shape.sig[0] / 2 - 2,
        x=ds.shape.sig[1] / 2 - 2,
    ),
    scan_rotation=330/180*np.pi,  # rad
    flip_y=False,
    semiconv=0.020,
)

In [5]:
keep_running = True
def on_click(btn):
    global keep_running
    keep_running = False
    btn.description = '👍 stopping'

stop_btn = ipywidgets.Button(description='Stop series')
stop_btn.on_click(on_click)

In [6]:
test_params = overfocus_params.derive()

In [7]:
overfocus_udf = OverfocusUDF(
    overfocus_params={'params': test_params},
)
sum_udf = SumUDF()

point_plot = BQLive2DPlot(dataset=ds, udf=overfocus_udf, channel='point')
shifted_sum_plot = BQLive2DPlot(dataset=ds, udf=overfocus_udf, channel='shifted_sum')
sum_plot_plain = BQLive2DPlot(dataset=ds, udf=sum_udf, channel='intensity')
sum_plot = BQLive2DPlot(dataset=ds, udf=overfocus_udf, channel='corrected_sum')
selected_plot = BQLive2DPlot(dataset=ds, udf=overfocus_udf, channel='selected')

plots = (point_plot, shifted_sum_plot, sum_plot_plain, selected_plot, sum_plot, )

adjustment_outputs = []

for p in plots:
    # Capture the plots to display them in a grid later
    output = ipywidgets.Output()
    with output:
        p.display()
        # Some plot-specific tweaks for grid display
        if isinstance(p, BQLive2DPlot):
            p.figure.fig_margin={'top': 50, 'bottom': 0, 'left': 25, 'right': 25}
            p.figure.layout.width = '250px'
            p.figure.layout.height = '250px'
    adjustment_outputs.append(output)

rp = test_params

# Create input and output widgets
scan_rotation = ipywidgets.FloatSlider(min=0, max=360, description='Scan rotation / deg', value=rp.scan_rotation * 180 / np.pi)
flip_y = ipywidgets.Checkbox(description='Flip detector Y axis', value=rp.flip_y)
overfocus = ipywidgets.FloatText(description='Overfocus / m', value=rp.overfocus)
camera_length = ipywidgets.FloatText(description='Camera length / m', value=rp.camera_length)
detector_pixel_pitch = ipywidgets.FloatText(description='Detector pixel pitch / m', value=rp.detector_pixel_pitch)
scan_pixel_pitch = ipywidgets.FloatText(description='Scan pixel pitch / m', value=rp.scan_pixel_pitch)
blur = ipywidgets.FloatText(description='Blur metric', value=0, disabled=True)

# Functions to update params and widgets
def update_blur(udf_result):
    blur.value = blur_effect(udf_result['shifted_sum'].data)

def params_to_input(params: Parameters4DSTEM):
    tp = params
    scan_rotation.value = tp.scan_rotation * 180 / np.pi
    flip_y.value = tp.flip_y
    overfocus.value = tp.overfocus
    camera_length.value = tp.camera_length
    detector_pixel_pitch.value = tp.detector_pixel_pitch
    scan_pixel_pitch.value = tp.scan_pixel_pitch

def input_to_params(params: Parameters4DSTEM):
    return params.derive(
        scan_rotation=scan_rotation.value / 180 * np.pi,
        flip_y=flip_y.value,
        overfocus=overfocus.value,
        camera_length=camera_length.value,
        detector_pixel_pitch=detector_pixel_pitch.value,
        scan_pixel_pitch=scan_pixel_pitch.value,
    )

# Arrange and display widgets
ipywidgets.VBox([
    ipywidgets.HBox(adjustment_outputs[0:3]),
    ipywidgets.HBox(adjustment_outputs[3:6]),
    ipywidgets.HBox([scan_rotation, blur]),
    flip_y,
    ipywidgets.HBox([overfocus, camera_length]),
    ipywidgets.HBox([detector_pixel_pitch, scan_pixel_pitch]),
    stop_btn
])

/home/weber/miniforge3/envs/calib313/lib/python3.13/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


## Usage

First, adjust scan rotation, flip_y and overfocus in such a way that the "OverfocusUDF: selected" panel shows a recognizable feature of the object in the same size and orientation as "OverfocusUDF: point" panel. "OverfocusUDF: shifted_sum" should start showing a blurred version of "OverfocusUDF: point". Then optimize the parameters until "OverfocusUDF: shifted_sum" appears sharp and the blur metric is minimized.

You can stop the repeated processing with the "Stop series" button at the bottom of the controls.

In [8]:
with ui_events() as poll:
    keep_running = True
    stop_btn.description = 'Stop series'
    params_to_input(test_params)
    while keep_running:
        result_iter = ctx.run_udf_iter(dataset=ds, udf=(overfocus_udf, sum_udf), plots=plots)
        for adjustment_res in result_iter:
            test_params = input_to_params(test_params)
            result_iter.update_parameters_experimental([
                {'overfocus_params': {'params': test_params}},
                {}
            ])
            poll(100)
        update_blur(adjustment_res.buffers[0])
    stop_btn.description = 'Stopped'

/home/weber/miniforge3/envs/calib313/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:4842: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
/home/weber/miniforge3/envs/calib313/lib/python3.13/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


## Numerical optimization

After adjusting the parameters manually, you can further refine the result with numerical optimization. This also updates the plots and input elements of the live plot above.

Please note that it doesn't work well with the simulated example data. The experimental dataset works better.

In [9]:
def my_callback(args, params: Parameters4DSTEM, res, blur):
    print(args, 'loss', blur, 'scan_rotation', params.scan_rotation, 'overfocus', params.overfocus)
    update_blur(res[0])
    params_to_input(params)

make_new_params, loss = make_overfocus_loss_function(
    params=test_params,
    ctx=ctx,
    dataset=ds,
    overfocus_udf=overfocus_udf,
    extra_udfs=(sum_udf, ),
    callback=my_callback,
    plots=plots
)
opt_res = optimize(loss)
print(opt_res)
test_params = make_new_params(opt_res.x)
print(test_params)

[0. 0.] loss 0.3251188127863435 scan_rotation 0.6422811647339132 overfocus 0.0011
[-10.  10.] loss 0.3642994017983773 scan_rotation 0.46774823953448025 overfocus 0.0013750000000000001
[-10. -10.] loss 0.34058030022904745 scan_rotation 0.46774823953448025 overfocus 0.000825
[10. 10.] loss 0.36948632440339296 scan_rotation 0.8168140899333463 overfocus 0.0013750000000000001
[ 10. -10.] loss 0.3364744575253445 scan_rotation 0.8168140899333463 overfocus 0.000825
[0. 0.] loss 0.3251188127863435 scan_rotation 0.6422811647339132 overfocus 0.0011
[1. 0.] loss 0.32506825710670967 scan_rotation 0.6597344572538565 overfocus 0.0011
[1. 1.] loss 0.3289710491129588 scan_rotation 0.6597344572538565 overfocus 0.0011275
[ 1.01295263 -0.99991611] loss 0.32267609875391734 scan_rotation 0.6599605233735276 overfocus 0.0010725023069444624
[ 1.03408585 -1.99969278] loss 0.32022004883999544 scan_rotation 0.6603293674890975 overfocus 0.0010450084485536875
[ 1.02847262 -2.99967703] loss 0.3204355866234841 scan_r

## Show the beam path in the TEMGYM Basic model

The model is scaled to make the beam path recognizable.

In [10]:
exaggerated_params = test_params.derive(
    scan_pixel_pitch=1e2*test_params.scan_pixel_pitch,
    overfocus=1e2*test_params.scan_pixel_pitch,
)
print(exaggerated_params)
model = Model4DSTEM.build(params=exaggerated_params, scan_pos=PixelYX(y=0, x=0))
plot_params = PlotParams(extent_scale=1.1)
fig, ax = plot_model(model.components, plot_params=plot_params)

Parameters4DSTEM(overfocus=9.999999999999999e-05, scan_pixel_pitch=9.999999999999999e-05, scan_center=PixelYX(y=32.0, x=32.0), scan_rotation=np.float64(0.6690558762670594), camera_length=0.15, detector_pixel_pitch=5e-05, detector_center=PixelYX(y=30.0, x=30.0), semiconv=0.02, flip_y=False, descan_error=DescanError(pxo_pxi=0.0, pxo_pyi=0.0, pyo_pxi=0.0, pyo_pyi=0.0, sxo_pxi=0.0, sxo_pyi=0.0, syo_pxi=0.0, syo_pyi=0.0, offpxi=0.0, offpyi=0.0, offsxi=0.0, offsyi=0.0), detector_rotation=0.0)


NameError: name 'PlotParams' is not defined

In [ ]:
fig.savefig('visualization.svg')